**Herzlich Willkommen !! **

I welcome you guys to my Deep learning tutorial. This tutorial is based for those guys who are
beginners or moderators and want to learn how to write modular code which is efficient. We will be using the Dog vs Cat dataset to do binary classification using Vanilla CNN.

Full code is available [hier](https://github.com/Gnopal1132/DogsVsCat).
I hope it helps you guys if you do hit star at my blog ;)


![Img](https://user-images.githubusercontent.com/51056214/131360401-03ff9c24-a589-4113-93e0-d37037d20692.png)

Every code specially Deep learning code must be divided into basic modules which are:
    
* **Configuration module** : This module is the core module of the whole model. It defines the full ingredient that will be used in the model like: hyperparameter values,dataset path,..

* **Generator Module** : As you guys have already heard that how important it is to preprocess the data !!(If not boy you are in for a suprise) so the data generators should be given a seperate module. This module will generate the data for our model

* **Preprocessing Module** : This module contains all the preprocessing that is needed to do like scaling,augmentations etc

* **Model Module** : This module is defined for creating the different models like in our example we will build COnvolutional Neural Network, we could define VGG16 or Fully convolutional module or whatever but it should be given a sperate module

* **Trainer Module** : This module is the one that trains the model with the configurations defined in the configuration files and saves the result.

* **Predictor Module** : This module predicts the output on test set

* **Main Module** : This module which connects all the model together this is the main.py

We will see every component in detail. 

**OK Lets jump right into it ;)**

**1. Configuration module** : This is the root file. Imagine u need to specify the batch_size parameter
 to be used by the model. This parameter might be tuned at various different place so to get your world
not messy we will define all the configuration at one place and use it from the configuration file.

In [ ]:
---
dataset:
  path: '/home/Dokumente/DogsVsCat/Datasets/dogs-vs-cats'
  train_size: 0.8
  size_x: 224
  size_y: 224
  classes: 2
  channel: 3
  path_image: '/home/Dokumente/DogsVsCat/Generated/dataset.png'
  predict_image: '/home/Dokumente/DogsVsCat/Generated/prediction.png'
train:
  optimizer: NADAM       # Possible values: ADAM,NADAM,SGD,SGD_MOMENTUM,RMS_PROP,ADA_GRAD
  learning_rate: 0.001
  batch_size: 32
  use_multiprocessing: True
  num_workers: -1
  epochs: 1
  weight_initialization:
    use_pretrained: False
    restore_from: '/home/Dokumente/DogsVsCat/Generated/last.h5'
  output_weight: '/home/Dokumente/DogsVsCat/Generated/final.h5'
network:
  graph_path: '/home/Dokumente/DogsVsCat/Generated/graph.json'
  model_img: '//home/Dokumente/DogsVsCat/Generated/model.png'
data_aug:
  use_aug: False
callbacks:
  earlystopping:
    use_early_stop: True
    patience: 10
    monitor: 'val_loss'
  checkpoint:
    checkpoint_last:
      enabled: True
      monitor: 'val_loss'
      out_last: '/home/Dokumente/DogsVsCat/Generated/last.h5'
    checkpoint_best:
      enabled: True
      monitor: 'val_loss'
      out_last: '/home/Dokumente/DogsVsCat/Generated/best.h5'
  tensorboard:
    enabled: True
    log_dir: '/home/Dokumente/DogsVsCat/Generated/logs'
  scheduler:
    onecycle:
      to_use : True
      max_rate: 0.05
    exponential_scheduler:
      to_use : False
      params: 10

In [ ]:
This is how a configuration file looks like. Lets Dig it down Shall we ;)

1. This configuration file is defined as yaml file. (i.e. cofig.yaml)
2. Its very similar to json file or to make life simple its very similar to a simple python
   dictionary.
    
for example:
    config['dataset']['path'] gives you the path where the dataset is located.
    
To access any parameter just follow the tree structure path.
for example: If I want to access the patience parameter of earlystopping I will go as follows:
        config['callbacks']['earlystopping']['patience'] = 10

See its Easy Peasy ;)

Lets see what we will be using:
    1. Dataset: This is pretty clear. It simply defines the path of dataset,size of image we 
        will be using. To look at what our dataset looks like and prediction looks like we
        define two different paths to store image of our dataset and predictions given by
        path_image and predict_image.
    2.Train: This defines the working configuration. Like which optimizer to use,learning rate,
        etc. Weight initialization is used for defining if I want to use last trained weights
        to initialize our model or to start from scratch.
        
        if config['train']['weight_initialization']['use_pretrained']:
            model.load_weight(config['train']['weight_initialization']['restore_from'])
            
        YOU KNOW WHAT I MEAN RIGHT ?
        
        Likewise output_weight stores the final weights.
        
    3. Network: This stores our model graph(What is that ? we will see) and our model image
    
    4.data_aug: Whether to use data augmentation. We will define several option
    
    5.Callbacks: Very important. Callbacks is something that is executed (before/after) every
        (epoch/batch) to do something. For example we define earlystopping checkpoint it will
        monitor validation loss if it doesnt improve for patience number of epochs we will stop.
        
        ModelCheckpoint to save the (best/last) model configuration till yet.
        onecycle scheduler or exponential scheduler : to change the value of learning rate after
            every (batch/epoch). Onecycle was introduced by Leslie Smith in 2017 paper. Very
            popular and powerful. We will see ;) 
        
        Wether to use them or not is defined by the switch parameter 'to_use'.

2. **Generator Module** : I just cant say how important it is. This module will give the model a batch of data to process. There are different ways of defining it like:
* **Tensorflow Data API**: One of my favorites but I wont introduce this topic here. 
* **Keras Data Generator**: Also one of my favorites easy and beautiful.


Lets Have a Look at keras Data Generator.

In [ ]:
import sys
import os
import tensorflow as tf
import numpy as np
from Preprocessing.preprocessing import read_image,read_image_test
#sys.path.append(os.path.dirname(os.path.abspath(os.curdir)))


class Datagenerator(tf.keras.utils.Sequence):
    def __init__(self, config, dataset, shuffle=True, is_train=True):
        self.config = config
        self.dataset = dataset
        self.is_train = is_train
        self.len_dataset = len(dataset)
        self.indices = np.arange(self.len_dataset)
        self.shuffle = shuffle
        self.classes = config['dataset']['classes']
        self.aug = config['data_aug']['use_aug']
        self.x_size = config['dataset']['size_x']
        self.y_size = config['dataset']['size_y']
        self.channels = config['dataset']['channel']
        self.batch_size = config['train']['batch_size']

        if self.shuffle:
            self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):   # Returns number of batches per epoch
        return int(np.floor(self.len_dataset/self.batch_size))

    def __getitem__(self, index):  # Generated batch for given index

        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        id = [self.dataset[k] for k in indices]
        if self.is_train:
            x, y = self.__data_generation(id)
            return x, y
        else:
            x = self.__data_generation(id)
            return x

    def __data_generation(self, ids):

        x_batch = []
        y_batch = []
        if self.is_train:
            for instance in ids:
                image, label = read_image(instance, size=(self.x_size, self.y_size), to_aug=self.aug)
                x_batch.append(image)
                y_batch.append(label)
            x_batch = np.asarray(x_batch, dtype=np.float32)
            y_batch = np.asarray(y_batch, dtype=np.float32)
            return x_batch, y_batch
        else:
            batch = []
            for img in ids:
                image = read_image_test(img, size=(self.x_size, self.y_size))
                batch.append(image)
            return np.asarray(batch)


In [ ]:
I know it seems Big but its small actually its tiny :hahah

Lets break it down shall we:
    
    1. To define keras data generator. You have to subclass(inherit) the tf.keras.utils.Sequence
    class.
    
    2. def __init__(self, config, dataset, shuffle=True, is_train=True): Is our constructor,
        config is our configuration file, dataset is (train/val/test) set , shuffle means
        to whether shuffle the data: For test set it will be false. is_train: Whether the 
        dataset is (train/val) or test set. if test set then it will be false and for rest
        it will be true.
        
    3. Inside the constructor everything should be clear. You can see here how we are using the
    configuration file. ALWAYS USE CONFIGURATION FILE. NEVER DO self.batch_size = 32. THATS
    BAD PRACTISE ALWAYS USE CONFIGURATION FILE. BECAUSE LATER IF YOU WANT TO CHANGE BATCH SIZE
    YOU HAVE TO ONLY CHANGE CONFIGURATION FILE AND REST WILL AUTOMATICALLY BE CHANGED BECUASE
    THEY ALL BE USING CONFIGURATION FILE.
    
    4. on_epoch_end(): This function is invoked once in constructor for (train/val) not for test.
        This function will automatically be invoked after every epoch ends. In this we will simply
        shuffle the dataset. Note: np.random.shuffle() to shuffle dataset inplace it doesnt
        return anything.
    
    5. def __len__(self):   Returns number of batches per epoch
        
    6.def __getitem__(self, index): It Generates a batch for given index. If you see here the
        indices actually stores the indices of the instances forming a batch and id will store
        those datapoints(here actually dataset is the addresses of the train/test/val set) and
        if its train/val: we have to return the instance and its label.
        if test set: we have to return only the instances.
        
    7. def __data_generation(self, ids): VERY CORE MODULE. This does the core part. It takes the
        ids and returns the batch. The inside code is very easy. You will notice that it calls
        read_image function for (train/val) set and read_image_test for test set. We will see this
        function just in  a while. But for now just understand that it will read the image,apply
        scaling, augmentation if needed and returns the file image and label. For test image we
        simply read it scale it but no data augmentation

3. **Preprocessing Module:** Now lets see how this read_image and read_image_test us defined.

In [ ]:
import numpy as np
import cv2
import os
from pathlib import Path
import albumentations as A


def read_image(path, size, to_aug=False):
    path = Path(path)
    if not path.exists():
        raise Exception("Image Not Found")
    else:
        image = cv2.imread(str(path))
        # By default cv2 reads in Blue,Green,Red Format(BGR) to convert in RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        image = cv2.resize(image, size)
        if to_aug:
            image = Augment_me(image)
        image = scaled_img(image)
        label = 0 if 'cat' in str(os.path.basename(path)) else 1
        return image, label


def read_image_test(path, size):
    path = Path(path)
    if not path.exists():
        raise Exception("Image Not Found")
    else:
        image = cv2.imread(str(path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # By default cv2 reads in Blue,Green,Red Format to convert in RGB
        image = cv2.resize(image, size)
        image = scaled_img(image)
        return np.asarray(image)


def scaled_img(image):
    image = np.asarray(image)
    mean = np.mean(image, axis=0, keepdims=True)
    return (image - mean)   # For images we dont in practise divide by std


def Augment_me(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.RandomRotate90(p=0.5)
    ])
    return transform(image=image)["image"]

I think there is nothing to explain here everything is very simple and self explanatory.

Some few Points: 
* [Albumentation](https://albumentations.ai/): This is one of the standard API for data augmentation it has a lot of options I used just few of them. A.compose: It defines a sequential stack of augmentations, here I used HorizontalFlip,RandomBrightness.. p = probability that the augmentation will take place. SIMPLE !! Nothing to explain here I guess! If anything is not clear please write in comment section.
       
* Note that when we will pass the dataset in Generator it will be simply the addresses of
  the instances in train/test/val set. So we read them bring them into right format. for
  labels: the address contains the name of the image as well like: ./dogsvscat/cat123.png
  So when I write **os.path.basename()** it will return the last path of directory
  i.e.cat123.png and after converting to string I find the right label. 0: Cat,1:Dog.
    
    
 * And we see whether to apply augmentation or not
    

At this Point Our data module is Complete !!! Man time for small **Coffee** ;)

OK After coffee lets move further to our 

**4. Model Module**

In [ ]:
import tensorflow as tf
from functools import partial

class Models:
    def __init__(self, config):
        self.config = config
        self.x_size = config['dataset']['size_x']
        self.y_size = config['dataset']['size_y']
        self.channels = config['dataset']['channel']
        self.model_img = config['network']['model_img']
        self.learning_rate = config['train']['learning_rate']
        self.optimizer = config['train']['optimizer']

    def convolution_scratch(self, save_model=False):
        tf.keras.backend.clear_session()
        Default = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1, padding='same', activation='relu')
        model = tf.keras.models.Sequential([
            Default(filters=32, kernel_size=7, strides=2, input_shape=[self.x_size, self.y_size, self.channels]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=2),

            Default(filters=64),
            Default(filters=64),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.MaxPool2D(pool_size=2),

            Default(filters=128),
            Default(filters=128),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.MaxPool2D(pool_size=2),

            Default(filters=256),
            Default(filters=256),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.MaxPool2D(pool_size=2),

            Default(filters=512),
            Default(filters=512),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.MaxPool2D(pool_size=2),

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(300, activation='relu', use_bias=False),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')  # Becuase its Binary CLassification
        ])

        if save_model:
            tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, show_layer_names=True, to_file=self.model_img)

        if self.optimizer == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, beta_1=0.9, beta_2=0.999)
        elif self.optimizer == 'NADAM':
            optimizer = tf.keras.optimizers.Nadam(learning_rate=self.learning_rate, beta_1=0.9, beta_2=0.999)
        elif self.optimizer == 'SGD':
            optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        elif self.optimizer == 'SGD_MOMENTUM':
            optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate, momentum=0.9)
        elif self.optimizer == 'RMS_PROP':
            optimizer = tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate, rho=0.9)
        elif self.optimizer == 'ADA_GRAD':
            optimizer = tf.keras.optimizers.Adagrad(learning_rate=self.learning_rate)
        else:
            raise Exception('Optimizer properly not defined in Configuration!!')

        model.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=optimizer, metrics=['accuracy'])
        return model

In [ ]:
I think its pretty clear!! Nothing to explain much but still lets me try:
    1. Model class simply takes the configuration file. And Intialize the things we need.
    2. def convolution_scratch: Defines our main model. partial is a simple tool that
        will make your life easy or else you have to write whole line. Just define the the 
        basic function as partial with default parameters to use and later just change the 
        parameter and rest will automatically be taken from default param.
        
        We used BatchNormalization() and Dropout
        
    3. You can also define another function in this class like def VGG16_Arch() and define
    your own architecture. Here is where you can show what you got!! haha
    
    4. Then we plot the model graphically and store in directory if needed.
    5. We then define the optimizer according to our configuration file
    6. And we compile the model.
    
I GUESS YOU CAN SEE HOW IMPORTANT THIS CONFIGURATION FILE IS.

5. **Trainer Module**: This is the module where we will train our module define the callbacks 

In [ ]:
import tensorflow as tf
import os

K = tf.keras.backend


class one_cycle(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None, last_iteration=None, last_rate=None):
        super().__init__()
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iteration = last_iteration or iterations // 10 + 1
        self.half_iteration = (self.iterations - self.last_iteration) // 2
        self.last_rate = last_rate or max_rate / 1000
        self.iteration = 0
        self.loss = []
        self.rate = []

    def __interpolate(self, start_iteration, final_iteration, start_rate, final_rate):
        return ((final_rate - start_rate)*(self.iteration - start_iteration))/((final_iteration-start_iteration) + start_rate)

    def on_batch_begin(self, batch, logs=None):
        if self.iteration < self.half_iteration:
            rate = self.__interpolate(0, self.half_iteration,self.start_rate,self.max_rate)
        elif self.iteration < 2*self.half_iteration:
            rate = self.__interpolate(self.half_iteration, 2*self.half_iteration, self.max_rate, self.start_rate)
        else:
            rate = self.__interpolate(2*self.half_iteration, self.iterations,self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

    def on_epoch_end(self, epoch, logs=None):
        self.loss.append(logs['loss'])
        self.rate.append(K.get_value(self.model.optimizer.lr))


class exponential_scheduler(tf.keras.callbacks.Callback):
    def __init__(self, s=40000):
        super().__init__()
        self.s = s
        self.loss = []
        self.rate = []

    def on_batch_begin(self, batch, logs=None):
        lr = K.get_value(self.model.optimizer.lr)
        K.set_value(self.model.optimizer.lr, lr*0.001*(1/self.s))

    def on_epoch_end(self, epoch, logs=None):
        self.loss.append(logs['loss'])
        self.rate.append(K.get_value(self.model.optimizer.lr))


class Result_callback(tf.keras.callbacks.Callback):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.loss_epoch = []
        self.val_loss_epoch = []
        self.rate = []
        self.epoch = []

    def on_epoch_end(self, epoch, logs=None):
        self.rate.append(K.get_value(self.model.optimizer.lr))
        self.loss_epoch.append(logs['loss'])
        self.val_loss_epoch.append((logs['val_loss']))
        self.epoch.append(epoch)


class Trainer:
    def __init__(self, config, model, train_loader, val_loader):
        self.config = config
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.model = model
        self.batch_size = self.config['train']['batch_size']
        self.epochs = self.config['train']['epochs']
        self.results = Result_callback()
        self.callbacks = self.get_callbacks().append(self.results)

    def get_id(self, root_dir):
        import time
        id_ = time.strftime('run_id_%Y_%m_%D_%H_%M_%S')
        return os.path.join(root_dir, id_)

    def get_callbacks(self):

        callbacks = []
        if self.config['callbacks']['earlystopping']['use_early_stop']:
            patience = self.config['callbacks']['earlystopping']['patience']
            monitor = self.config['callbacks']['earlystopping']['monitor']
            early_stop = tf.keras.callbacks.EarlyStopping(patience=patience, monitor=monitor)
            callbacks.append(early_stop)

        if self.config['callbacks']['checkpoint']['checkpoint_last']['enabled']:
            monitor = self.config['callbacks']['checkpoint']['checkpoint_last']['monitor']
            file_path = self.config['callbacks']['checkpoint']['checkpoint_last']['out_last']
            checkpoint_last = tf.keras.callbacks.ModelCheckpoint(monitor=monitor,
                                                                 save_best_only=False,
                                                                 save_weights_only=True,
                                                                 filepath=file_path)
            callbacks.append(checkpoint_last)

        if self.config['callbacks']['checkpoint']['checkpoint_best']['enabled']:
            monitor = self.config['callbacks']['checkpoint']['checkpoint_best']['monitor']
            file_path = self.config['callbacks']['checkpoint']['checkpoint_best']['out_last']
            checkpoint_best = tf.keras.callbacks.ModelCheckpoint(monitor=monitor,
                                                                 save_best_only=True,
                                                                 save_weights_only=True,
                                                                 filepath=file_path)
            callbacks.append(checkpoint_best)

        if self.config['callbacks']['tensorboard']['enabled']:
            directory = self.get_id(self.config['callbacks']['tensorboard']['log_dir'])
            board = tf.keras.callbacks.TensorBoard(directory, write_graph=True, write_images=True)
            callbacks.append(board)

        if self.config['callbacks']['scheduler']['onecycle']['to_use']:
            iterations = self.epochs * self.batch_size
            max_rate = self.config['callbacks']['scheduler']['onecycle']['max_rate']
            one_cycle_callback = one_cycle(iterations=iterations, max_rate=max_rate)
            callbacks.append(one_cycle_callback)

        if self.config['callbacks']['scheduler']['exponential_scheduler']['to_use']:
            s = self.config['callbacks']['scheduler']['exponential_scheduler']['params']
            exponential_scheduler_callback = exponential_scheduler(s=s)
            callbacks.append(exponential_scheduler_callback)
        return callbacks

    def train(self):
        # Lets now train this bad boy

        if self.config['train']['weight_initialization']['use_pretrained']:
            reload_from = self.config['train']['weight_initialization']['restore_from']
            print(f"Restoring model weights from: {reload_from}")
            self.model.load_weights(reload_from)
        else:
            print(f"Saving graph in: {self.config['network']['graph_path']}")
            self.save_graph(self.model, self.config['network']['graph_path'])

        use_multiprocessing = self.config["train"]['use_multiprocessing']
        num_workers = self.config["train"]["num_workers"]

        self.model.fit_generator(generator=self.train_loader, validation_data=self.val_loader,
                                 callbacks=self.callbacks, epochs=self.epochs, use_multiprocessing=use_multiprocessing,
                                 workers=num_workers, shuffle=False, max_queue_size=10, verbose=1)

        print(f"Saving Weights in {self.config['train']['output_weight']}")
        self.model.save_weights(self.config['train']['output_weight'])

    def save_graph(self, model, path):
        model_json = model.to_json()
        with open(path, "w") as json_file:
            json_file.write(model_json)


OK TO BE HONEST THIS IS THE PART I HAVE TO EXPLAIN MY ASS OFF!!

Lets Break it down:
* **Callbacks**: There are some callbacks that you can use directly like Earlystop and Checkpoints. But tensorflow also have the option to define custom callbacks which I used here for Onecycle,Exponential Schedule and ResultCallback. How to define custom [callbacks](https://www.tensorflow.org/guide/keras/custom_callback) ? Ok let me break it down. 
  1. For creating custom callback you need to subclass tf.keras.callbacks.Callback class.
  2. Now you have predefined function like on_(train|test|predict)_begin(self, logs=None),
     on_(train|test|predict)_end(self, logs=None),on_(train|test|predict)_batch_begin(self, batch, logs=None),etc... (See the documentation above link) Now every function is either called at the (beginning/end) of (batch/epoch/train/test/predict) this depends on which you are using. All we gotta do is manipulate the function accordingly.
     
*      Result Callback: I stored the learning rate,epoch,loss and validation loss after every epoch. It will be used for analysis.
*     Exponential Scheduler: new_lr = prev_lr * 0.001**(1/s) the learning rate changes by this formula at beginning of every batch and I am also storing the results at end of one epoch.
*     Onecycle: This scheduler increases linearly the learning rate from lr0 to lr1 during first half of iteration and then decreases the learning rate from lr1 to lr0 during another half still linearly and during the last epochs it reduces the learning rate by several magnitude. max_rate defines the rate of increasing and decreasing the lr. Simply read the function properly you will understand.

**get_callbacks()** function returns the list of callbacks to use by reading the configuration file. Note model checkpoint stores the last checkpoint and the best checkpoint
found till now.
     

**train()** module first checks if I want to use previous trained weights to initialize the
model or start from scratch. If from scratch it first saves the model structure/graph in json
file first and then start the training. The model.fit_generator is similar to model.[fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model) the only difference is we are using generator module.And then we save the final weights

6. **Predictor Module**: This module is the one that predicts the test set. It first load the final model graph and then load the best found weights to that model and does the prediction.

In [ ]:
import tensorflow as tf
from Preprocessing.preprocessing import read_image_test
from Data_Generator.generator import Datagenerator


class Predictor:
    def __init__(self, config, test_path):
        self.config = config
        self.graph_path = self.config['network']['graph_path']
        self.model_weight = self.config['train']['output_weight']
        self.x_size = config['dataset']['size_x']
        self.y_size = config['dataset']['size_y']
        self.model = self.load_model()
        self.test_loader = Datagenerator(self.config, test_path, shuffle=False, is_train=False)

    def load_model(self):
        json_file = open(self.graph_path, 'r')
        load_json = json_file.read()
        json_file.close()

        model = tf.keras.models.model_from_json(load_json)
        model.load_weights(self.model_weight)
        return model

    def predict(self):
        class_predict = []
        predictions = self.model.predict(self.test_loader, batch_size=None)
        for prediction in predictions:
            if prediction >= 0.5:
                class_predict.append('Dog')
            else:
                class_predict.append('Cat')
        return class_predict

In [ ]:
Its pretty clear. We first define a generator for test set and call model.predict() on that
generator it will return the probability for every instance and if the probability is greater
than 50% its a Dog else Cat.

See how we first loaded the model graph from json we saved earlier and then we loaded the
final weights to do prediction

7. **Main Module**: This module is the main file that connects everthing

In [ ]:
import os
import warnings
from Prediction import predictor
from Models.vision_model import Models
from Data_Generator.generator import Datagenerator
import numpy as np
import matplotlib.pyplot as plt
import yaml
import cv2
from pathlib import Path
from Trainer import trainer


# This function simply takes the train/val/test set path and returns complete path to every instance in the set
def read_dataset(source_path, shuffle=True):
    files = os.listdir(source_path)  # Will read the directory content and return the name of files present inside
    preprocessed_paths = []
    for instance in files:
        preprocessed_paths.append(os.path.join(source_path, instance)) #Complete path
    preprocessed_paths = np.asarray(preprocessed_paths)
    if shuffle:
        np.random.shuffle(preprocessed_paths)  # It does inplace
    return preprocessed_paths

# To plot the predicted images 50 of them
def show_some_image_prediction(images, labels, path_to_save=None):

    n_rows = 10
    n_cols = 5

    plt.figure(figsize=(20, 20))
    for row in range(n_rows):
        for col in range(n_cols):
            idx = n_cols * row + col
            plt.subplot(n_rows, n_cols, idx + 1)
            img = cv2.imread(images[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            label = labels[idx]
            plt.imshow(img)
            plt.title(label, fontsize=12)
            plt.axis('off')
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    if path_to_save is not None:
        plt.savefig(path_to_save)
    plt.show()

# To plot the dataset images
def show_some_images(list_images, path_to_save=None):
    n_rows = 5
    n_cols = 5
    plt.figure(figsize=(10, 8))
    for row in range(n_rows):
        for col in range(n_cols):
            idx = n_cols*row + col
            plt.subplot(n_rows, n_cols, idx+1)
            img = cv2.imread(list_images[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            label = 'Cat' if 'cat' in str(os.path.basename(list_images[idx])) else 'Dog'
            plt.imshow(img)
            plt.title(label, fontsize=12)
            plt.axis('off')
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    if path_to_save is not None:
        plt.savefig(path_to_save)
    plt.show()

    
    
# Start from Here

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)


# Loading the COnfiguration FIle

config_file = Path(os.path.join(os.curdir, "Configuration", "config"))
if not config_file.exists():
    raise Exception("configuration missing!!")
else:
    with open(config_file) as f:
        config_file = yaml.load(f)

        
# Path to training set and TEst set
train_path = os.path.join(config_file["dataset"]["path"], 'train')
test_path = os.path.join(config_file["dataset"]["path"], 'test1')


# We read the train set to get the entire path to every instance in a list and we split
# to train and val set for test set we dont do any shuffle
data = read_dataset(train_path)
train_size = int(len(data)*config_file["dataset"]["train_size"])
train = data[:train_size]
val = data[train_size:]
test = read_dataset(test_path, shuffle=False)

show_some_images(train, config_file['dataset']['path_image'])  
#To see some dataset images and storing the image to directory


# We create the Generator for train and val set. The generator will give a batch of preprocessed
# data to the model to train
train_loader = Datagenerator(config_file, train, shuffle=True)
val_loader = Datagenerator(config_file, val, shuffle=True)

# We initialize the model by sending the configuration 
baseline_model = Models(config=config_file).convolution_scratch(save_model=True)

# We initialize the trainer class by sending the loader and config file
trainer = trainer.Trainer(config=config_file, model=baseline_model, train_loader=train_loader, val_loader=val_loader)
# We train the model here.
trainer.train()

# Prediction
predict = predictor.Predictor(config_file, test)
class_predict = predict.predict()

# To see the final predictions
show_some_image_prediction(test, class_predict, path_to_save=config_file['dataset']['predict_image'])

Below shows the output of last line. It gives around 95% of accuracy

![](https://github.com/Gnopal1132/DogsVsCat/blob/main/Generated/prediction.png?raw=true)

ONE FINAL NOTE MAKE SURE YOU SET UP THE DIRECTORY STURCTURE PROPERLY LIKE IF YOU SEE SOME 
IMPORTS I DID SOMETHING LIKE **from Data_Generator.generator import Datagenerator**.
IF DIRECTORY STRUCTURE IS NOT PROPERLY MANAGED IT WILL GIVE ERROR THAT FILE DOESNT EXIST.

Checkout full code at my [github](https://github.com/Gnopal1132/DogsVsCat)

So Finally I reached the END!! I hope u enjoyed as much as I did. Please star my blog here and on my github.

**Thank You**

**Ich freue mich auf deine Reaktion auf meinen Beitrag ;)**